## Job shop scheduling problem (JSSP) with mixed integer programming (MIP)

In [3]:
from ortools.linear_solver import pywraplp

$$
\begin{align}
    \text{min} \quad & C \\
    \text{s.t.} \quad & x_{\sigma_{h-1}^j, j} + p_{\sigma_{h-1}^j, j} \leq x_{\sigma_{h}^j, j}
        & \forall ~ j \in J; h \in (2, ..., |M|)\\
    & x_{m, j} + p_{m, j} \leq x_{m, k} + V (1 - z_{m, j, k})
        & \forall ~ j, k \in J, j \neq k; m \in M\\
    & z_{m, j, k} + z_{m, k, j} = 1
        & \forall ~ j, k \in J, j \neq k; m \in M\\
    & x_{\sigma_{|M|}^j, j} + p_{\sigma_{|M|}^j, j} \leq C
        & \forall ~ j \in J\\
    & x_{m, j} \geq 0 & \forall ~ j \in J; m \in M\\
    & z_{m, j, k} \in \{0, 1\} & \forall ~ j, k \in J; m \in M\\
\end{align}
$$

In [7]:
def jssp_MIP(durations: list[list[int]], machines: list[list[int]]):
    model = pywraplp.Solver("JSSP", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    machines_count = len(machines[0])
    jobs_count = len(machines)
    starts = {}
    all_jobs = range(jobs_count)
    all_machines = range(machines_count)
    horizon = sum([sum(duration) for duration in durations])
    """
    Khởi tạo biến bắt đầu thời gian thực hiện mỗi task trên mỗi máy, là các biến nguyên không âm.
    """
    for i in all_jobs:
        for j in all_machines:
            starts[(i, j)] = model.IntVar(0, horizon, "start_%i_%i" % (i, j))

    """
    Ràng buộc các task phải được thực hiện tuần tự.
    """
    for i in all_jobs:
        for j in range(machines_count - 1):
            model.Add(starts[(i, j)] + durations[i][j] <= starts[(i, j + 1)])

    """
    Ràng buộc mỗi máy chỉ thực hiện một task tại một thời điểm.
    """
    # Tìm tập các task được thực hiện trên mỗi máy.
    machine_to_jobs = {}
    for i in all_machines:
        machines_jobs = []
        for j in all_jobs:
            for k in all_machines:
                if machines[j][k] == i:
                    machines_jobs.append((j, k))
        machine_to_jobs[i] = machines_jobs
    # Tạo biến nhị phân để chỉ ra task nào được thực hiện trước task nào trên mỗi máy.
    precedence = {}
    for i in all_machines:
        for j in machine_to_jobs[i]:
            for k in machine_to_jobs[i]:
                if j[0] != k[0]:
                    precedence[(j, k)] = model.BoolVar(
                        "precedence_%i(%i)_%i(%i)" % (j + k)
                    )
    # Ràng buộc 2 task liên tiếp trên mỗi máy phải được thực hiện tuần tự (tổng các biến nhị phân = 1).
    for i in all_machines:
        for j in range(len(machine_to_jobs[i]) - 1):
            for k in range(j + 1, len(machine_to_jobs[i])):
                model.Add(
                    precedence[(machine_to_jobs[i][j], machine_to_jobs[i][k])]
                    + precedence[(machine_to_jobs[i][k], machine_to_jobs[i][j])]
                    == 1
                )
    # Với mỗi task thực hiện trước, gán ràng buộc về thời gian để các task không bị trùng thời gian.
    V = 1_000_000
    for i in all_machines:
        for j in machine_to_jobs[i]:
            for k in machine_to_jobs[i]:
                if j[0] != k[0]:
                    model.Add(
                        starts[j] + durations[j[0]][j[1]]
                        <= starts[k] + V * (1 - precedence[(j, k)])
                    )
    """
    Khởi tạo biến mục tiêu và ràng buộc.
    """
    obj_var = model.IntVar(0, horizon, "makespan")
    for i in all_jobs:
        model.Add(
            obj_var
            >= starts[(i, machines_count - 1)] + durations[i][machines_count - 1]
        )
    model.Minimize(obj_var)
    model.set_time_limit(60000)
    status = model.Solve()
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(
            "Optimal"
            if status == pywraplp.Solver.OPTIMAL
            else "Feasible" + " Schedule Length:",
            model.Objective().Value(),
        )
    else:
        print("No optimal solution found.")


if __name__ == "__main__":
    import numpy as np

    np.random.seed(24)
    # generate random data for durations and machines
    n_job = 10
    n_machine = 3
    durations = np.random.randint(1, 100, size=(n_job, n_machine)).tolist()
    machines = [np.random.permutation(n_machine).tolist() for _ in range(n_job)]
    print("Durations: ")
    for duration in durations:
        print(*duration)
    print("---" * n_machine)
    print("Machines: ")
    for machine in machines:
        print(*machine)
    print("---" * n_machine)
    jssp_MIP(durations, machines)

Durations: 
35 4 65
88 18 18
2 80 5
83 12 16
74 19 8
26 36 96
29 1 13
96 34 2
61 56 71
98 82 7
---------
Machines: 
0 1 2
0 1 2
2 0 1
1 2 0
0 2 1
2 1 0
1 0 2
2 0 1
0 2 1
2 0 1
---------
Feasible Schedule Length: 567.0
